<h2>Hadware</h2>

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

<h2> Liberias </h2>

In [1]:
import pandas as pd
import sys
import os
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation, Add
from tensorflow.python.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K
import PIL

from tensorflow.python.keras.applications.imagenet_utils import preprocess_input
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import datetime


<h3>Definimos las constantes, hiperparametros, etc</h3>

In [2]:
#K.image_data_format() CHANNELS_LAST
nb_train_samples = 7947 
nb_validation_samples = 300


train_dir = 'data/train'
dev_dir = 'data/dev'
test_dir = ' data/test'
epochs = 5
width, height = 306, 306
batch_size = 20
steps = nb_train_samples // batch_size
validation_steps = nb_validation_samples // batch_size
#Tamaño profundidad de filtros
filtersConv1 = 32
filtersConv2 = 32
filtersConv3 = 64

#Tamaño de los filtros
filter_size1 = (3,3)
filter_size2 = (3,3)
filter_size3 = (3,3)
#PoolSizes
pool_size1 = (2,2)
pool_size2 = (2,2)
pool_size3 = (2,2)

number_class = 3
learning_rate = 0.005
lr = learning_rate
#Limpiamos sesiones previas
K.clear_session()

filter_convs = [filtersConv1, filtersConv2, filtersConv3]
filter_sizes = [filter_size1, filter_size2, filter_size3]
pool_sizes   = [pool_size1, pool_size2, pool_size3]

 

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.2,0.8],
    rotation_range=20,
)
    

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(width, height),
    batch_size=batch_size,
    class_mode='categorical')
    
validation_generator = test_datagen.flow_from_directory(
    dev_dir,
    target_size=(width, height),
    batch_size=batch_size,
    class_mode='categorical')
    
 

Found 7953 images belonging to 3 classes.
Found 311 images belonging to 3 classes.


<h2>Modelo</h2>

In [ ]:
def model(input_shape, filter_convs, filter_sizes, pool_sizes):
    width, height = input_shape[0], input_shape[1] 
    if K.image_data_format() == 'channels_first':
        input_shape = (3, width, height)
    else:
        input_shape = (width, height, 3)

    network =Sequential()
    print(input_shape)
    #first layer
    network.add(Convolution2D( filter_convs[0],
                               filter_sizes[0], 
                               padding='same', 
                               input_shape = input_shape,
                               activation='relu'
                             ))
    network.add(MaxPooling2D(pool_size=pool_sizes[0]))
    
    #Second layer
    network.add(Convolution2D( filter_convs[1],
                               filter_sizes[1],  
                               activation='relu'
                             ))
    network.add(MaxPooling2D(pool_size=pool_sizes[1]))
    
    #third layer
    network.add(Convolution2D( filter_convs[2],
                               filter_sizes[2], 
                               activation='relu'
                             ))
    
    network.add(MaxPooling2D(pool_size=pool_sizes[2]))
    
    
    network.add(Flatten())
    network.add(Dense(64))
    network.add(Activation('relu'))
    network.add(Dropout(0.5))
    network.add(Dense(3))
    network.add(Activation('softmax'))



    
    return network

In [ ]:
input_shape = (width, height)
cnn = model(input_shape, filter_convs, filter_sizes, pool_sizes )
#Primer modelo
#cnn.compile(loss='mean_squared_error', optimizer='Adam', metrics = ['accuracy'])
#CategoricalCrossentropy
cnn.compile(loss='categorical_crossentropy', optimizer='Adam', metrics = ['accuracy'])


<h2>Entrenamiento del modelo</h2>

Una vez entrenado se guarda el modelo

In [ ]:
cnn.fit_generator(
        train_generator,
        steps_per_epoch=steps,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size,
        use_multiprocessing=True,
        workers = 4
    )
#Una vez que se entrena, el modelo se guardara en el directorio especificado
dir = 'modelo2/'+ str(datetime.datetime.now()) + '/'
if os.path.exists(dir) == False:
    os.mkdir(dir)
#os.path.exists(dir)
cnn.save(dir + 'modelo.h5')
cnn.save_weights(dir  +'weights.h')


In [ ]:
dir = 'modelo2/'+ str(datetime.datetime.now()) + '/'
if os.path.exists(dir) == False:
    os.mkdir(dir)
#os.path.exists(dir)
cnn.save(dir + 'modelo.h5')
cnn.save_weights(dir  +'weights.h')

<h2>Cargamos el modelo pre entrenado</h2> 


In [ ]:
loaded_model= tf.keras.models.load_model('modelo/2020-02-07 00:27:13.124754/modelo.h5')

In [ ]:
loaded_model.fit_generator(
        train_generator,
        steps_per_epoch=steps,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size,
        use_multiprocessing=True,
        workers = 4
)
dir = 'modelo2/'+ str(datetime.datetime.now()) + '/'
if os.path.exists(dir) == False:
    os.mkdir(dir)
#os.path.exists(dir)
loaded_model.save(dir + 'modelo.h5')
loaded_model.save_weights(dir  +'weights.h')


In [ ]:
results = pd.read_csv('test.csv')
results

In [ ]:
from PIL import Image as img
images_set = []
for index, row in results.iterrows():
   # print(row['Id'])
    data = img.open('Selfie-dataset/images/' + str(row['Id']) + '.jpg')
    images_set.append(np.asarray(data))

In [ ]:
rst = loaded_model.predict(np.array(images_set))
np.argmax(rst, axis = 1)


In [ ]:
def changeIndex(results):
    rta = []
    for i in range(len(results)):
        #print(results[i])
        if(results[i] == 0):
            rta.append(2)
        elif results[i] == 1:
            rta.append(0)
        else:
            rta.append(1)
    return rta
            

In [ ]:
train_generator.class_indices

In [ ]:
rta = changeIndex(np.argmax(rst, axis = 1))
results['Category'] =  rta
results

In [ ]:
results.to_csv('results.csv')

In [ ]:
results.hist(figsize=(15,5))

In [ ]:
test_generator = test_datagen('PATH_TO_DATASET_DIR/Dataset',
                              # only read images from `test` directory
                              #classes=['test'],
                              # don't generate labels
                              #class_mode=None,
                              # don't shuffle
                              shuffle=False,
                              # use same size as in training
                              target_size=(306, 306))

In [4]:
loaded_model= tf.keras.models.load_model('modelo2/2020-02-07 13:48:29.638705/modelo.h5')
loaded_model.summary()
loaded_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics = ['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 306, 306, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 153, 153, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 151, 151, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 73, 73, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 82944)             0

In [5]:
loaded_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics = ['accuracy'])
try:
    loaded_model.fit_generator(
            train_generator,
            steps_per_epoch=steps,
            epochs=epochs,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples // batch_size,
            use_multiprocessing=True,
            workers = 4
        )
except tf.errors.ResourceExhaustedError as e:
    #Una vez que se entrena, el modelo se guardara en el directorio especificado
    dir = 'modelo2/'+ str(datetime.datetime.now()) + '/'
    if os.path.exists(dir) == False:
        os.mkdir(dir)
    #os.path.exists(dir)
    loaded_model.save(dir + 'modelo.h5')
    loaded_model.save_weights(dir  +'weights.h')


Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 397 steps, validate for 15 steps
Epoch 1/5
397/397 [==============================] - 190s 480ms/step - loss: 0.5472 - accuracy: 0.7597 - val_loss: 0.5010 - val_accuracy: 0.7633
Epoch 2/5
397/397 [==============================] - 170s 428ms/step - loss: 0.5374 - accuracy: 0.7626 - val_loss: 0.4608 - val_accuracy: 0.7600
Epoch 3/5
397/397 [==============================] - 171s 432ms/step - loss: 0.5376 - accuracy: 0.7653 - val_loss: 0.6038 - val_accuracy: 0.7200
Epoch 4/5
397/397 [==============================] - 169s 427ms/step - loss: 0.5276 - accuracy: 0.7674 - val_loss: 0.4928 - val_accuracy: 0.7900
Epoch 5/5
397/397 [==============================] - 172s 433ms/step - loss: 0.5310 - accuracy: 0.7644 - val_loss: 0.4796 - val_accuracy: 0.7700


In [6]:
dir = 'modelo2/'+ str(datetime.datetime.now()) + '/'
if os.path.exists(dir) == False:
    os.mkdir(dir)
    #os.path.exists(dir)
loaded_model.save(dir + 'modelo.h5')
loaded_model.save_weights(dir  +'weights.h')